## Sentiment Values
- textblob

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

In [3]:
path = 'yelp_data/health_text_full.csv'

df = pd.read_csv(path)
df.drop(['Unnamed: 0'],axis=1,inplace=True)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64006 entries, 0 to 64005
Data columns (total 14 columns):
business_id        64006 non-null object
categories         64006 non-null object
name               64006 non-null object
state              64006 non-null object
categories_list    64006 non-null object
cool               64006 non-null int64
date               64006 non-null object
funny              64006 non-null int64
review_id          64006 non-null object
stars              64006 non-null int64
text               64006 non-null object
useful             64006 non-null int64
user_id            64006 non-null object
clean_text         64006 non-null object
dtypes: int64(4), object(10)
memory usage: 6.8+ MB


In [5]:
df.head()

,business_id,categories,name,state,categories_list,cool,date,funny,review_id,stars,text,useful,user_id,clean_text
0,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2018-03-03,0,TNNkSmMfshsD3G60jTNjDA,1,Please stay away from this place if you can! I...,2,xv2V2GO5IZYvtw4oW7gQ1w,please stay away place bad care imaginable sta...
1,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2015-11-29,0,v-iKdstPdCxJr8zV1ZMdrw,5,My husband has been a patient of Dr. Byrne for...,1,SjvWP7c9toeZoV_q62zhTA,husband patient dr byrne last year half last m...
2,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2016-06-03,0,BmNDRCV9_NzQ_KCChyfdEw,4,Dr. Byrne is a great doctor! She has great bed...,2,sZVHm1aLtvyH9trAc2_MgA,dr byrne great doctor great bed side manner ex...
3,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2017-03-17,0,t_TKVMxKFYm9Hl-TIO7UUw,3,I'm raising my review as Dr Bryne's has been m...,3,8Y_irXocZdZxLs_qgzpjBw,raise review dr bryne receptive daughter go an...
4,y-4xTZNKVm8mAZpiXMS5ZA,"urologists, doctors, health & medical","Lauren Byrne, MD",AZ,"['urologists', ' doctors', ' health & medical']",0,2016-08-31,0,NWRrpGRgWZBBj3lvCZGVKA,1,I wish I could give 0 stars. Worst office I've...,1,hVKPDGpG12z7vpScXaSakw,wish could give star bad office ever horrible ...


## Textblob

In [7]:
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk import tokenize

In [17]:
def sentiment_value(tweet):
    """ vater function get value """
    anlyzr = SentimentIntensityAnalyzer()
    # break review into sentences and analyze each seperately
    content = tokenize.sent_tokenize(tweet)
    # emtpy values
    overall = {'compound': 0, 'neg': 0, 'neu': 0, 'pos': 0}
    for s in content:
        scores = anlyzr.polarity_scores(s)
        # tally up each sentence's overall tone
        for i, z in enumerate(scores):
            overall[z] += scores[z]
    # average it all together for the review as a whole
    for v in overall:
        overall[v] = round(overall[v] / len(content), 3)
    return overall

In [21]:
%%time
original = df.text.apply(sentiment_value).apply(pd.Series)

CPU times: user 13min 10s, sys: 1min 38s, total: 14min 48s
Wall time: 14min 49s


In [127]:
%%time
clean = df.clean_text.apply(sentiment_value).apply(pd.Series)

CPU times: user 11min 37s, sys: 1.22 s, total: 11min 38s
Wall time: 11min 38s


In [157]:
ls yelp_data/

health_business00.csv*  health_text_model_results.csv*
health_business01.csv*  health_text_sentiment.csv*
health_raw00.csv*       health_text_sentiment_full.csv*
health_text.csv*        yelp_academic_dataset_business.json*
health_text_full.csv*   yelp_academic_dataset_review.json*


In [132]:
og = pd.merge(df[['review_id','stars','text']], original,right_index=True,left_index=True)

cl = pd.merge(df[['review_id','stars','clean_text']], clean,right_index=True,left_index=True)

In [158]:
og.to_csv('yelp_data/sent_text.csv')

In [159]:
cl.to_csv('yelp_data/sent_text_clean.csv')